# Detecting Baby Face from CCTV footage

In [ ]:
%pip install ultralytics moviepy opencv-python torch torchvision torchaudio

# Add parent folder to sys.path and import config
import sys, os
sys.path.append(os.path.abspath('..'))
import config as cfg

# Expose commonly used variables with safe fallbacks
DATA_ROOT = getattr(cfg, 'DATA_ROOT', "/kaggle/input/listening-beyond-the-cry/Final Dataset")
DATA_DIR = getattr(cfg, 'DATA_DIR', DATA_ROOT)
MODEL_PATH = getattr(cfg, 'CRY_TYPE_WEIGHTS', getattr(cfg, 'CRY_TYPE_WEIGHTS', 'best_infantcry_model.pth'))
TEST_AUDIO = getattr(cfg, 'TEST_AUDIO', None)
INPUT_VIDEO1 = getattr(cfg, 'INPUT_VIDEO1', None)
YOLO_WEIGHTS = getattr(cfg, 'YOLO_WEIGHTS', None)

print('Config loaded from parent folder:', os.path.abspath(os.path.join('..','config.py')))
print('DATA_ROOT:', DATA_ROOT)
print('MODEL_PATH:', MODEL_PATH)
print('INPUT_VIDEO1:', INPUT_VIDEO1)


In [ ]:
!pip install roboflow ultralytics supervision moviepy

from roboflow import Roboflow
rf = Roboflow(api_key="tmlb5en1LBu6OosgU9FL")
project = rf.workspace("sleepingbaby").project("baby-project")
version = project.version(1)

dataset = version.download("yolov11", location="roboflow_dataset")
print("Dataset downloaded to:", dataset.location)


In [ ]:
import os

def filter_labels(folder, keep_classes=[0]):
    for split in ["train", "valid", "test"]:
        lbl_dir = os.path.join(folder, split, "labels")
        if not os.path.exists(lbl_dir):
            print("WARNING: missing labels folder →", lbl_dir)
            continue

        for file in os.listdir(lbl_dir):
            if not file.endswith(".txt"):
                continue

            path = os.path.join(lbl_dir, file)
            with open(path, "r") as f:
                lines = f.read().strip().splitlines()

            new_lines = []
            for line in lines:
                if line.strip() == "":
                    continue

                parts = line.split()
                if len(parts) < 5:
                    continue  # skip invalid YOLO lines

                try:
                    cls = int(parts[0])
                except:
                    continue

                if cls in keep_classes:
                    parts[0] = "0"  # map to single class
                    new_lines.append(" ".join(parts))

            # Write updated labels back
            with open(path, "w") as f:
                if new_lines:
                    f.write("\n".join(new_lines))

print("Filtering labels...")
filter_labels("roboflow_dataset", keep_classes=[0])
print("Done: Kept only class 0 (face).")

In [ ]:
data_yaml = """\
path: roboflow_dataset

train: train/images
val: valid/images
test: test/images

nc: 1
names:
  0: face
"""
with open("data.yaml", "w") as f:
    f.write(data_yaml)

print("Correct data.yaml written.")


In [ ]:
import cv2

root = "roboflow_dataset"
bad_files = []

for sub in ["train", "valid", "test"]:
    img_dir = os.path.join(root, sub, "images")
    if not os.path.exists(img_dir):
        print("WARNING: missing images folder →", img_dir)
        continue

    for f in os.listdir(img_dir):
        path = os.path.join(img_dir, f)
        img = cv2.imread(path)
        if img is None:
            bad_files.append(path)
            print("CORRUPT:", path)

print("\nTotal corrupt files:", len(bad_files))

In [ ]:
import os

root = "roboflow_dataset/train/images"
bad = []
zero = []

for f in os.listdir(root):
    path = os.path.join(root, f)
    size = os.path.getsize(path)
    if size == 0:
        zero.append(path)
    if size < 500:  # suspiciously tiny
        bad.append((path, size))

len(zero), len(bad), zero[:5], bad[:5]


In [ ]:
import os
import cv2

root = "roboflow_dataset"

valid_ext = {".jpg", ".jpeg", ".png"}

bad_files = []

for split in ["train", "valid", "test"]:
    img_dir = os.path.join(root, split, "images")
    if not os.path.exists(img_dir):
        print("Missing:", img_dir)
        continue

    for f in os.listdir(img_dir):
        path = os.path.join(img_dir, f)

        # Check extension
        ext = os.path.splitext(f)[1].lower()
        if ext not in valid_ext:
            print("⚠ Removing non-image:", path)
            os.remove(path)
            continue

        # Check readable image
        img = cv2.imread(path)
        if img is None:
            print("❌ Removing corrupt image:", path)
            os.remove(path)
            bad_files.append(path)

print("\nRemoved corrupt files:", len(bad_files))


In [ ]:
import os

root = "roboflow_dataset"

for split in ["train", "valid", "test"]:
    lbl_dir = os.path.join(root, split, "labels")
    if not os.path.exists(lbl_dir):
        continue

    for f in os.listdir(lbl_dir):
        path = os.path.join(lbl_dir, f)
        if os.path.getsize(path) == 0:
            print("⚠ Removing empty label:", path)
            os.remove(path)


In [ ]:
import os
import cv2

def is_image(path):
    try:
        img = cv2.imread(path)
        return img is not None
    except:
        return False

root = "roboflow_dataset"

bad_files = []

for split in ["train", "valid", "test"]:
    folder = os.path.join(root, split)
    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)

        # skip labels directory
        if os.path.isdir(path):
            continue

        # check for valid image extensions
        if not fname.lower().endswith((".jpg",".jpeg",".png",".bmp",".tif",".tiff")):
            bad_files.append(path)
            os.remove(path)
            continue

        # check if image opens properly
        if not is_image(path):
            bad_files.append(path)
            os.remove(path)

print("Removed bad files:")
for f in bad_files:
    print(f)

print("Cleaning done.")


In [ ]:
import os

root = "roboflow_dataset"

for split in ["train", "valid", "test"]:
    labels = os.path.join(root, split, "labels")
    for fname in os.listdir(labels):
        if not fname.endswith(".txt"):
            print("⚠️ Removing non-txt file inside labels:", fname)
            os.remove(os.path.join(labels, fname))


In [ ]:
import os
import cv2
import numpy as np

ROOT = "/kaggle/working/roboflow_dataset"

def safe_read(path):
    try:
        with open(path, "rb") as f:
            data = f.read()
        arr = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
        return img
    except Exception as e:
        print("⚠️ Exception while reading", path, ":", e)
        return None

bad_files = []

print("🔍 Scanning ALL image files YOLO may attempt to load...\n")

for split in ["train", "valid", "test"]:
    img_dir = os.path.join(ROOT, split, "images")
    if not os.path.exists(img_dir):
        print("MISSING FOLDER:", img_dir)
        continue

    for file in os.listdir(img_dir):
        full = os.path.join(img_dir, file)
        if not file.lower().endswith((".jpg",".jpeg",".png",".bmp",".tif",".tiff")):
            continue

        img = safe_read(full)
        if img is None:
            print("❌ CORRUPTED IMAGE:", full)
            bad_files.append(full)
        else:
            h, w = img.shape[:2]
            if h == 0 or w == 0:
                print("❌ ZERO-SIZE IMAGE:", full)
                bad_files.append(full)

print("\n============================")
print("FINISHED SCANNING")
print("============================\n")

if bad_files:
    print("❌ BAD FILES FOUND:")
    for b in bad_files:
        print("→", b)
else:
    print("✅ No corrupted images found.")



In [ ]:
print(len(bad_files))

In [ ]:
from ultralytics import YOLO

print("Loading YOLO model...")
model = YOLO("yolo11n.pt")

print("Starting training...")
model.train(
    data="data.yaml",
    epochs=40,
    imgsz=640,
    batch=16,
    device=0,    # GPU on Kaggle
    amp=False    # STABLE FIX FOR KAGGLE AMP BUG
)

print("Training completed.")


In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics.utils.plotting import plot_results
from IPython.display import Image, display

print("RESULTS")

#combined results
results_dir = model.results_dir
if results_dir is None:  # fallback for older versions
    results_dir = "runs/detect/train"

train_results_file = os.path.join(results_dir, "results.png")

if os.path.exists(train_results_file):
    print("\nTraining Curves:")
    display(Image(filename=train_results_file))
else:
    print("results.png not found, manually plotting...")
    try:
        plot_results(file=os.path.join(results_dir, "results.csv"))
        display(Image(filename=train_results_file))
    except:
        print("Could not generate training curves.")

#last validation batch predictions
val_batch_img = os.path.join(results_dir, "val_batch0_pred.jpg")
if os.path.exists(val_batch_img):
    print("\nValidation Predictions (val_batch0_pred.jpg):")
    display(Image(filename=val_batch_img))
else:
    print("val_batch0_pred.jpg not found.")

#validation labels vs predictions
val_labels_img = os.path.join(results_dir, "val_batch0_labels.jpg")
if os.path.exists(val_labels_img):
    print("\nValidation Labels:")
    display(Image(filename=val_labels_img))
else:
    print("val_batch0_labels.jpg not found.")


#PR curve
pr_curve_file = os.path.join(results_dir, "PR_curve.png")
if os.path.exists(pr_curve_file):
    print("\nPrecision-Recall Curve:")
    display(Image(filename=pr_curve_file))
else:
    print("PR_curve.png not found.")

#F1 curve
f1_curve_file = os.path.join(results_dir, "F1_curve.png")
if os.path.exists(f1_curve_file):
    print("\nF1 Curve:")
    display(Image(filename=f1_curve_file))
else:
    print("F1_curve.png not found.")

#P curve
p_curve_file = os.path.join(results_dir, "P_curve.png")
if os.path.exists(p_curve_file):
    print("\nPrecision Curve:")
    display(Image(filename=p_curve_file))
else:
    print("P_curve.png not found.")

#SHOW R curve
r_curve_file = os.path.join(results_dir, "R_curve.png")
if os.path.exists(r_curve_file):
    print("\nRecall Curve:")
    display(Image(filename=r_curve_file))
else:
    print("R_curve.png not found.")

In [ ]:
# TESTING: process a sample video using config paths
import cv2
import moviepy.editor as mp
from ultralytics import YOLO
import os

input_video = INPUT_VIDEO1 
wav_output = "audio.wav"           # extracted audio file
silent_video = "video_no_audio.mp4"
final_output = "output_with_box.mp4"

print("Extracting audio...")
clip = mp.VideoFileClip(input_video)
clip.audio.write_audiofile(wav_output)
print("Saved:", wav_output)

print("Removing audio...")
clip_no_audio = clip.without_audio()
clip_no_audio.write_videofile(silent_video, codec='libx264', audio=False)
print("Saved:", silent_video)

# use YOLO weights from config if available
yolo_model_path = YOLO_WEIGHTS if YOLO_WEIGHTS else "yolo11n.pt"
print('Using YOLO model:', yolo_model_path)
model = YOLO(yolo_model_path)

cap = cv2.VideoCapture(silent_video)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(final_output, fourcc, fps, (width, height))

print("Processing video...")

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO prediction
    results = model(frame, conf=0.5, verbose=False)

    # Draw boxes
    for r in results:
        boxes = getattr(r.boxes, 'xyxy', None)
        if boxes is None:
            continue
        for box in boxes:
            x1, y1, x2, y2 = map(int, box[:4])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

    out.write(frame)

    frame_idx += 1
    if frame_idx % 50 == 0:
        print("Processed frame:", frame_idx)

cap.release()
out.release()

print("Saved final video:", final_output)


In [ ]:
from IPython.display import HTML

HTML(f"""
<video width="700" controls>
  <source src="{final_output}" type="video/mp4">
</video>
""")


# Checking if Baby is Crying

In [ ]:
import os
import glob
import random
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import Dataset, DataLoader, random_split


In [ ]:
# Use DATA_ROOT from config (set earlier)
# DATA_ROOT = "/kaggle/input/listening-beyond-the-cry/Final Dataset"

print('DATA_ROOT in use:', DATA_ROOT)

cry_paths = glob.glob(os.path.join(DATA_ROOT, "Cry", "**", "*.wav"), recursive=True)
notcry_paths = glob.glob(os.path.join(DATA_ROOT, "Not-cry", "**", "*.wav"), recursive=True)

print("Cry:", len(cry_paths))
print("Not Cry:", len(notcry_paths))

# balance cry = notcry
num_notcry = len(notcry_paths)
if num_notcry == 0:
    raise RuntimeError(f"No 'Not-cry' audio files found under DATA_ROOT={DATA_ROOT}. Update DATA_ROOT in config.py or the notebook.")
cry_paths_balanced = random.sample(cry_paths, num_notcry)

all_files = [(p, 1) for p in cry_paths_balanced] + [(p, 0) for p in notcry_paths]
random.shuffle(all_files)

print("Balanced total:", len(all_files))


In [ ]:
class BabyCryDataset(Dataset):
    def __init__(self, file_label_pairs, fixed_frames=224):
        self.data = file_label_pairs
        self.fixed_frames = fixed_frames

        self.mel_tf = torchaudio.transforms.MelSpectrogram(
            sample_rate=16000,
            n_fft=1024,
            hop_length=512,
            n_mels=64
        )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filepath, label = self.data[idx]

        # Load audio
        wav, sr = torchaudio.load(filepath)
        wav = torchaudio.functional.resample(wav, sr, 16000)
        wav = wav.mean(dim=0, keepdim=True)

        # Mel Spectrogram
        mel = self.mel_tf(wav)  # shape = [1, 64, T]

        # Normalize
        mel = (mel - mel.mean()) / (mel.std() + 1e-6)

        # === FORCE FIXED SHAPE ===
        # Interpolate along time dimension to exactly fixed_frames
        mel = F.interpolate(mel.unsqueeze(0), size=(64, self.fixed_frames), mode='bilinear', align_corners=False)
        mel = mel.squeeze(0)

        return mel, torch.tensor(label, dtype=torch.float32)


In [ ]:
dataset = BabyCryDataset(all_files)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)


In [ ]:
class CryClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = self.net(x)
        x = x.view(x.size(0), -1)
        return torch.sigmoid(self.fc(x)).squeeze()


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CryClassifier().to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for mel, y in train_loader:
        mel, y = mel.to(device), y.to(device)

        pred = model(mel)
        loss = loss_fn(pred, y)

        opt.zero_grad()
        loss.backward()
        opt.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss:.4f}")

print("Training complete!")


In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for mel, y in val_loader:
        mel = mel.to(device)
        preds = model(mel)

        preds = (preds.cpu().numpy() > 0.5).astype(int)
        y_pred.extend(preds)
        y_true.extend(y.numpy().astype(int))

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
cm = confusion_matrix(y_true, y_pred)

print("\n=== Evaluation Metrics ===")
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("\nConfusion Matrix:\n", cm)


In [ ]:
def predict_wav(path):
    model.eval()

    wav, sr = torchaudio.load(path)
    wav = torchaudio.functional.resample(wav, sr, 16000)
    wav = wav.mean(dim=0, keepdim=True)

    mel_tf = torchaudio.transforms.MelSpectrogram(
        sample_rate=16000, n_fft=1024, hop_length=512, n_mels=64
    )
    mel = mel_tf(wav)
    mel = (mel - mel.mean()) / (mel.std() + 1e-6)

    mel = mel.unsqueeze(0).to(device)   # batch=1

    with torch.no_grad():
        pred = model(mel).item()

    if pred > 0.5:
        return f"Cry ({pred:.3f})"
    else:
        return f"Laugh ({pred:.3f})"


In [ ]:
save_path = "notcry_cry_classifier.pth"
torch.save(model.state_dict(), save_path)

print("Model saved to:", save_path)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#Confusion Matrix Heatmap

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Not Cry", "Cry"],
            yticklabels=["Not Cry", "Cry"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

#Bar Chart of Metrics
metrics = ["Accuracy", "Precision", "Recall", "F1"]
values = [acc, prec, rec, f1]

plt.figure(figsize=(7, 5))
plt.bar(metrics, values)
plt.ylim(0, 1)
plt.title("Model Performance Metrics")
plt.ylabel("Score")
plt.show()

#Random Mel Spectrogram Samples from Validation set
import random
from IPython.display import Audio

print("\nShowing 3 random validation samples:")
for _ in range(3):
    mel, label = random.choice(val_ds)
    plt.figure(figsize=(8, 3))
    plt.imshow(mel.squeeze().numpy(), aspect='auto', origin='lower')
    plt.title(f"Label: {'Cry' if label==1 else 'Not Cry'}")
    plt.colorbar()
    plt.show()

#Prediction Distribution Histogram

all_probs = []

model.eval()
with torch.no_grad():
    for mel, label in val_loader:
        mel = mel.to(device)
        probs = model(mel).cpu().numpy()
        all_probs.extend(probs)

plt.figure(figsize=(7, 5))
plt.hist(all_probs, bins=30)
plt.title("Prediction Probability Distribution")
plt.xlabel("Predicted Probability (Cry)")
plt.ylabel("Count")
plt.show()



In [ ]:
print(predict_wav("/kaggle/input/test-crying/dunstan-s-baby-language-the-5-cries_Sfxjiev4.wav"))


# Classifying Different Types of Cries

In [ ]:
!pip install audiomentations==0.27.0

In [ ]:
import os, random, math, warnings, sys
from pathlib import Path
from collections import Counter
import numpy as np, pandas as pd
import librosa, librosa.display
import soundfile as sf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.models as models
import torch.optim as optim
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

In [ ]:
# DATA_DIR / DATA_ROOT use config-backed values
DATA_DIR = getattr(cfg, 'DATA_DIR', DATA_ROOT)

DATA_DIR = DATA_DIR  # dataset root
SR = 16000
DURATION = 7.0
N_MELS = 128
MFCC_N = 40
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 4
LEARNING_RATE = 1e-4
K_FOLDS = 5
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
print('Using DATA_DIR:', DATA_DIR)


In [ ]:
audio_ext = {".wav", ".mp3", ".flac", ".ogg"}
rows = []

root = Path(DATA_DIR)

if not root.exists():
    print("ERROR: DATA_DIR does not exist:", DATA_DIR)
    sys.exit(1)

# Iterate over folders
for label_dir in root.iterdir():
    if label_dir.is_dir():
        label = label_dir.name.strip().lower()
        for f in label_dir.iterdir():
            if f.suffix.lower() in audio_ext:
                rows.append({"path": str(f), "label": label})

df = pd.DataFrame(rows)

if df.empty:
    print("No audio files found. Check DATA_DIR.")
    sys.exit(1)

In [ ]:
augmenter = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.9, max_rate=1.1, p=0.4),
    PitchShift(min_semitones=-2, max_semitones=2, p=0.5),
    Shift(min_fraction=-0.1, max_fraction=0.1, p=0.5),
])

In [ ]:
def load_audio(path, sr=SR, duration=DURATION):
    y, orig_sr = sf.read(path)
    if len(y.shape) > 1:
        y = np.mean(y, axis=1)
    if orig_sr != sr:
        y = librosa.resample(y.astype(np.float32), orig_sr, sr)

    target_len = int(sr * duration)
    if len(y) < target_len:
        y = np.pad(y, (0, target_len - len(y)))
    else:
        y = y[:target_len]
    return y

In [ ]:
def compute_log_mel(y, sr=SR):
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    log_mel = librosa.power_to_db(mel, ref=np.max)
    return log_mel

In [ ]:
def compute_mfcc_features(y, sr=SR):
    mf = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=MFCC_N)
    return np.concatenate([mf.mean(axis=1), mf.std(axis=1)])

In [ ]:
class InfantCryDataset(Dataset):
    def __init__(self, df, augment=False, spec_augment=False):
        self.df = df.reset_index(drop=True)
        self.augment = augment
        self.spec_augment = spec_augment

        labels = sorted(self.df.label.unique())
        self.label2idx = {l:i for i,l in enumerate(labels)}
        self.idx2label = {i:l for l,i in self.label2idx.items()}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        y = load_audio(row.path)

        if self.augment:
            y = augmenter(samples=y, sample_rate=SR)

        log_mel = compute_log_mel(y)
        mfcc_feat = compute_mfcc_features(y)

        # normalize
        log_mel = (log_mel - log_mel.mean()) / (log_mel.std() + 1e-9)

        mel_tensor = torch.tensor(log_mel, dtype=torch.float).unsqueeze(0)
        mfcc_tensor = torch.tensor(mfcc_feat, dtype=torch.float)

        label_idx = self.label2idx[row.label]
        return mel_tensor, mfcc_tensor, label_idx

In [ ]:
class ResNetMel(nn.Module):
    def __init__(self, pretrained=True, out_dim=512):
        super().__init__()
        base = models.resnet18(pretrained=pretrained)
        base.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.base = nn.Sequential(*list(base.children())[:-1])
        self.embedding_dim = base.fc.in_features
        self.head = nn.Sequential(
            nn.Linear(self.embedding_dim, out_dim),
            nn.ReLU(),
            nn.BatchNorm1d(out_dim)
        )

    def forward(self, x):
        x = self.base(x)
        x = x.reshape(x.size(0), -1)
        return self.head(x)

In [ ]:
class FusionClassifier(nn.Module):
    def __init__(self, emb_dim=512, mfcc_dim=MFCC_N*2, num_classes=5):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(emb_dim + mfcc_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    def forward(self, emb, mfcc):
        return self.fc(torch.cat([emb, mfcc], dim=1))

In [ ]:
def train_one_epoch(cnn, fusion, optimizer, dl, criterion):
    cnn.train(); fusion.train()
    total = correct = loss_sum = 0

    for mel, mfcc, labels in dl:
        mel, mfcc, labels = mel.to(DEVICE), mfcc.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        emb = cnn(mel)
        logits = fusion(emb, mfcc)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item() * labels.size(0)
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return loss_sum/total, correct/total

In [ ]:
def eval_model(cnn, fusion, dl, criterion):
    cnn.eval(); fusion.eval()
    preds_list, label_list = [], []
    loss_sum = 0

    with torch.no_grad():
        for mel, mfcc, labels in dl:
            mel, mfcc, labels = mel.to(DEVICE), mfcc.to(DEVICE), labels.to(DEVICE)
            emb = cnn(mel)
            logits = fusion(emb, mfcc)
            loss = criterion(logits, labels)

            loss_sum += loss.item()*labels.size(0)
            preds_list.append(logits.argmax(1).cpu().numpy())
            label_list.append(labels.cpu().numpy())

    preds = np.concatenate(preds_list)
    labels = np.concatenate(label_list)
    return loss_sum/len(labels), preds, labels

In [ ]:
def run_kfold(df):
    skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED)
    X, y = df['path'].values, df['label'].values
    fold_reports = []

    for fold, (train_i, val_i) in enumerate(skf.split(X, y), 1):
        print(f"\n--- Fold {fold}/{K_FOLDS} ---")

        df_train = df.iloc[train_i]
        df_val   = df.iloc[val_i]

        ds_train = InfantCryDataset(df_train, augment=True)
        ds_val   = InfantCryDataset(df_val, augment=False)

        # balanced sampling
        counts = df_train.label.value_counts().to_dict()
        weights = df_train.label.map(lambda x: 1/counts[x]).values
        sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

        dl_train = DataLoader(ds_train, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4)
        dl_val   = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

        num_classes = len(ds_train.label2idx)

        cnn = ResNetMel(pretrained=True).to(DEVICE)
        fusion = FusionClassifier(num_classes=num_classes).to(DEVICE)

        # class weights
        class_counts = np.array([counts[l] for l in sorted(counts)])
        w = 1/class_counts
        w = torch.tensor(w/w.sum()*len(w), dtype=torch.float).to(DEVICE)
        criterion = nn.CrossEntropyLoss(weight=w)

        optimizer = optim.Adam(list(cnn.parameters()) + list(fusion.parameters()), lr=LEARNING_RATE)

        best_f1, best_state = -1, None

        for epoch in range(1, NUM_EPOCHS+1):
            tr_loss, tr_acc = train_one_epoch(cnn, fusion, optimizer, dl_train, criterion)
            val_loss, preds, labs = eval_model(cnn, fusion, dl_val, criterion)

            rpt = classification_report(labs, preds, output_dict=True)
            macro_f1 = rpt["macro avg"]["f1-score"]

            print(f"Epoch {epoch}: train_acc={tr_acc:.3f} val_f1={macro_f1:.3f}")

            if macro_f1 > best_f1:
                best_f1 = macro_f1
                best_state = {
                    "cnn": cnn.state_dict(),
                    "fusion": fusion.state_dict(),
                    "idx2label": ds_train.idx2label
                }

        fold_reports.append(best_state)
        print("Best Macro F1:", best_f1)

    return fold_reports


In [ ]:
if __name__ == "__main__":
    print("Starting training...")
    reports = run_kfold(df)
    torch.save(reports[-1], "best_infantcry_model.pth")
    print("Saved best model.")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

print("\n=== Generating Outputs ===")

# -----------------------------------------
# 1. SHOW PER CLASS SAMPLE COUNTS
# -----------------------------------------
plt.figure(figsize=(6,4))
df['label'].value_counts().plot(kind='bar')
plt.title("Number of Samples per Class")
plt.xlabel("Class")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# -----------------------------------------
# 2. SHOW SOME MELSPECTROGRAM IMAGES
# -----------------------------------------
print("\nShowing mel-spectrograms from few random audio files...")

sample_df = df.sample(3, random_state=42)

for i, row in sample_df.iterrows():
    y = load_audio(row.path)
    mel = compute_log_mel(y)

    plt.figure(figsize=(6,4))
    librosa.display.specshow(mel, sr=SR, x_axis='time', y_axis='mel')
    plt.title(f"Mel Spectrogram - {row.label}")
    plt.colorbar()
    plt.tight_layout()
    plt.show()

# -----------------------------------------
# 3. RUN MODEL ON FULL DATA & GET CONF MATRIX
# -----------------------------------------
print("\nRunning final model for confusion matrix & classification report...")

# load best (last fold) model
best = reports[-1]
idx2label = best["idx2label"]
label2idx = {v:k for k,v in idx2label.items()}

num_classes = len(idx2label)

cnn = ResNetMel(pretrained=False).to(DEVICE)
fusion = FusionClassifier(num_classes=num_classes).to(DEVICE)

cnn.load_state_dict(best["cnn"])
fusion.load_state_dict(best["fusion"])

cnn.eval()
fusion.eval()

all_preds = []
all_labels = []

for mel, mfcc, label in DataLoader(
        InfantCryDataset(df, augment=False),
        batch_size=16, shuffle=False):

    mel, mfcc = mel.to(DEVICE), mfcc.to(DEVICE)

    with torch.no_grad():
        emb = cnn(mel)
        logits = fusion(emb, mfcc)

    preds = logits.argmax(1).cpu().numpy()
    labels = label.numpy()

    all_preds.append(preds)
    all_labels.append(labels)

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# -----------------------------------------
# 4. CONFUSION MATRIX
# -----------------------------------------
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=idx2label.values(),
            yticklabels=idx2label.values())
plt.title("Confusion Matrix (Full Dataset)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

# -----------------------------------------
# 5. CLASSIFICATION REPORT
# -----------------------------------------
print("\n=== Classification Report ===")
print(classification_report(all_labels, all_preds, target_names=list(idx2label.values())))


In [ ]:
import torch
import torch.nn as nn
import librosa
import soundfile as sf
import numpy as np

# --------------------------
# CONFIG
# --------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SR = 16000
DURATION = 7.0
N_MELS = 128
MFCC_N = 40
# MODEL_PATH and TEST_AUDIO should come from config if available
MODEL_PATH = MODEL_PATH  # from config import cell
TEST_AUDIO = TEST_AUDIO or "/kaggle/input/baby-cry-sounds/Baby Dataset/tired/249c.wav"

# --------------------------
# FEATURE EXTRACTORS
# --------------------------
def load_audio(path, sr=SR, duration=DURATION):
    y, orig_sr = sf.read(path)
    if len(y.shape) > 1:
        y = np.mean(y, axis=1)
    if orig_sr != sr:
        y = librosa.resample(y.astype(np.float32), orig_sr, sr)
    target_len = int(sr * duration)
    if len(y) < target_len:
        y = np.pad(y, (0, target_len - len(y)))
    else:
        y = y[:target_len]
    return y

def compute_log_mel(y, sr=SR):
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    log_mel = librosa.power_to_db(mel, ref=np.max)
    log_mel = (log_mel - log_mel.mean()) / (log_mel.std() + 1e-9)
    return log_mel

def compute_mfcc_features(y, sr=SR):
    mf = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=MFCC_N)
    return np.concatenate([mf.mean(axis=1), mf.std(axis=1)])

# --------------------------
# MODEL DEFINITION (same as training)
# --------------------------
import torchvision.models as models

class ResNetMel(nn.Module):
    def __init__(self, pretrained=True, out_dim=512):
        super().__init__()
        base = models.resnet18(pretrained=pretrained)
        base.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.base = nn.Sequential(*list(base.children())[:-1])
        self.embedding_dim = base.fc.in_features
        self.head = nn.Sequential(
            nn.Linear(self.embedding_dim, out_dim),
            nn.ReLU(),
            nn.BatchNorm1d(out_dim),
        )
    def forward(self, x):
        x = self.base(x)
        x = x.reshape(x.size(0), -1)
        return self.head(x)

class FusionClassifier(nn.Module):
    def __init__(self, emb_dim=512, mfcc_dim=MFCC_N*2, num_classes=5):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(emb_dim + mfcc_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes),
        )
    def forward(self, emb, mfcc):
        return self.fc(torch.cat([emb, mfcc], dim=1))

# --------------------------
# LOAD MODEL
# --------------------------
checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
idx2label = checkpoint.get('idx2label', None)
if idx2label is None:
    # try older key names
    idx2label = checkpoint.get('label_map', None) or {i: l for i, l in enumerate(['class0','class1','class2','class3','class4'])}

cnn = ResNetMel(pretrained=False, out_dim=512).to(DEVICE)
fusion = FusionClassifier(emb_dim=512, mfcc_dim=MFCC_N*2, num_classes=len(idx2label)).to(DEVICE)

cnn.load_state_dict(checkpoint['cnn'])
fusion.load_state_dict(checkpoint['fusion'])
cnn.eval(); fusion.eval()

# --------------------------
# INFERENCE FUNCTION
# --------------------------
def predict(audio_path):
    y = load_audio(audio_path)
    log_mel = compute_log_mel(y)
    mfcc_feat = compute_mfcc_features(y)

    mel_tensor = torch.tensor(log_mel, dtype=torch.float).unsqueeze(0).unsqueeze(0).to(DEVICE)
    mfcc_tensor = torch.tensor(mfcc_feat, dtype=torch.float).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        emb = cnn(mel_tensor)
        logits = fusion(emb, mfcc_tensor)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred_idx = int(probs.argmax())
        pred_label = idx2label.get(pred_idx, str(pred_idx))
    return pred_label, probs

# --------------------------
# RUN ON TEST AUDIO
# --------------------------
if TEST_AUDIO is None:
    print('TEST_AUDIO not set in config; skipping sample predict step.')
else:
    label, probs = predict(TEST_AUDIO)
    print("Predicted label:", label)
    print("Class probabilities:", probs)


In [ ]:
import copy

def predict_tta(audio_path, n_augment=5):
    y_orig = load_audio(audio_path)
    probs_list = []
    for _ in range(n_augment):
        y = copy.deepcopy(y_orig)
        # apply augmentation
        y = augmenter(samples=y, sample_rate=SR)
        log_mel = compute_log_mel(y)
        mfcc_feat = compute_mfcc_features(y)
        mel_tensor = torch.tensor(log_mel, dtype=torch.float).unsqueeze(0).unsqueeze(0).to(DEVICE)
        mfcc_tensor = torch.tensor(mfcc_feat, dtype=torch.float).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            emb = cnn(mel_tensor)
            logits = fusion(emb, mfcc_tensor)
            probs_list.append(torch.softmax(logits, dim=1).cpu().numpy()[0])
    avg_probs = np.mean(probs_list, axis=0)
    pred_idx = avg_probs.argmax()
    pred_label = idx2label[pred_idx]
    return pred_label, avg_probs

label, probs = predict_tta("/kaggle/input/test-hunger/dunstan-s-baby-language-the-5-cries_Sfxjiev4.wav")
print("Predicted label (TTA):", label)
print("Class probabilities:", probs)
